# Yelp Data Merging

In [1]:
# Copied and pasted from my imports from project 3, 
# delete if we dont need(Aerika)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB, ComplementNB, GaussianNB, BernoulliNB

from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.tokenize import ToktokTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords 

import regex as re

from scipy.stats import ttest_ind

In [2]:
file_paths = []
for file in os.listdir('../data'):
    if 'businesses2019' in file:
        file_paths.append('../data/'+file)

In [3]:
master_df = {
    'id': [],
    'latitude': [],
    'longitude': [],
    'price': [],
    'review_count': [],
    'rating': [],
    'zip_code': [],
    'city': [],
    'alias': [],
    'category': [],
}

master_df = pd.DataFrame(master_df)

In [4]:
for path in file_paths:
    
    master_df = pd.concat([master_df, pd.read_csv(path)])

master_df.head()

C:\Users\m6zha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
0,rays-texas-bbq-huntington-park,"['bbq', 'smokehouse', 'salad']",Huntington Park,spKUIFIABLaU63Ex4PEYvQ,33.986598,-118.229803,$$,4.5,736.0,90255.0
1,the-grilled-cheese-truck-los-angeles,"['foodtrucks', 'tradamerican']",Los Angeles,lwBbzIzX1q7MavJOR7Tzyg,33.979080,-118.250090,$,3.5,1580.0,90001.0
2,avilas-el-ranchito-mexican-restaurant-huntingt...,['mexican'],Huntington Park,RYpJyi1RhScLZAYrqPMLKQ,33.978496,-118.230545,$$,4.5,576.0,90255.0
3,tokyo-doggie-style-los-angeles-6,"['foodtrucks', 'japanese', 'hotdog']",Los Angeles,PyivZWrHyYv4JQd9qGGrjw,33.978828,-118.249931,$$,4.5,180.0,90001.0
4,the-los-angeles-banh-mi-company-huntington-park,"['vietnamese', 'sandwiches', 'coffee']",Huntington Park,iKn3cdBPI9o2KNAJ4OHvLQ,33.974622,-118.228026,$,5.0,209.0,90255.0


In [5]:
# Seeing shape of dataframe BEFORE dropping duplicates 

master_df.shape

(73995, 10)

In [6]:
# Dropping duplicates

master_df = master_df.sort_values(['alias',
                       'id', 'latitude', 'longitude', 'review_count'],
                      ascending = False).drop_duplicates(subset = ['id', 'alias', 'zip_code'],
                                                         keep ='first')

In [7]:
# Shape of df AFTER dropping duplicates

master_df.shape

(10077, 10)

In [8]:
master_df.head()

,alias,category,city,id,latitude,longitude,price,rating,review_count,zip_code
26,いざかや-おせん-izakaya-osen-los-angeles,"['sushi', 'izakaya', 'seafood']",Los Angeles,us0WnDOySVXXXwCqs0AaCw,34.083192,-118.273245,$$,4.5,651.0,90026.0
463,ô-banh-mi-los-angeles,"['vietnamese', 'sandwiches', 'coffee']",Los Angeles,XRbyfQZ7nWwJCOjRC8EW-g,34.098450,-118.275100,$,4.0,111.0,90027.0
79,à-bloc-los-angeles,"['coffee', 'sandwiches', 'breakfast_brunch']",Los Angeles,fRDvAW7nE0-21QzqxTeC8w,34.121570,-118.205860,$,5.0,75.0,90042.0
32,zzamong-los-angeles,"['korean', 'chinese', 'noodles']",Los Angeles,gWM5n5Tyo5h74t-_lp1WUA,34.069115,-118.307394,$,4.0,859.0,90004.0
8,zweet-cafe-los-angeles,"['coffee', 'cafes', 'breakfast_brunch']",Los Angeles,9yB5H8MwuQ3MBMlq5EOoFw,34.129740,-118.216450,$,4.5,481.0,90041.0
